In [1]:
# Импорт библиотек
import pandas as pd
import os
import random
from math import pi
import torch
from torch.utils.data import random_split, Dataset, DataLoader, TensorDataset
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import filedialog

/tmp/ipykernel_52017/2001088470.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# #! Сначала склеить разметку, а потом уже определить разрывы и разметить цифрой 2 начала предупреждающих участков
root = tk.Tk()
root.withdraw()

filePath = filedialog.askopenfilename(filetypes=[("Parquet Files", "*.parquet")])
if filePath:
    print("Выбранный файл:", filePath)
    fileName, fileExtension = os.path.splitext(filePath)
    if fileExtension == '.parquet':
        dfOriginal = pd.read_parquet(filePath)
        df = dfOriginal.copy()
    else:
        print("Выбран неверный файл")
        exit()
else:
    print("Выбор файла отменен.")
    exit()

df['value'] = (df['value'] - df['value'].min()) / (df['value'].max() - df['value'].min())       # Нормализация данных
df.reset_index(drop=True)

Выбранный файл: /home/vest/Desktop/Work/NeuralNetworks/ecg-anomaly-detection/.workspace/Resources/SecondDone/ECGr1.C.S.V.parquet


,value,edge,time,group
0,0.190297,0.0,0.0,NaN
1,0.164994,0.0,1.0,NaN
2,0.177645,0.0,2.0,NaN
3,0.194514,0.0,3.0,NaN
4,0.160777,0.0,4.0,NaN
...,...,...,...,...
608595,0.000000,0.0,608595.0,NaN
608596,0.000000,0.0,608596.0,NaN
608597,0.000000,0.0,608597.0,NaN
608598,0.000000,0.0,608598.0,NaN


In [18]:
# class SecondEcgDataset(Dataset):
#     def __init__(self, df):
#         self.df = df
#         self.segmentStarts = df.index[df['edge'] > 0].tolist()
#         self.startsCount = len(self.segmentStarts)

#         self.cachedTensors = []
#         self._get_segment_tensor()
#         self.segmentsCount = len(self.cachedTensors)
    
#     def _get_segment_tensor(self):
#         for index in range(self.startsCount):
#             result = 0
#             if index + 1 < len(self.segmentStarts):
#                 if self.df.at[self.segmentStarts[index], 'edge'] < 2:
#                     segment = self.df.iloc[self.segmentStarts[index]:self.segmentStarts[index+1]]
#                     result = 1
#             elif self.df.at[self.segmentStarts[index], 'edge'] < 2:
#                 segment = self.df.iloc[self.segmentStarts[index]:]
#                 result = 1

#             if result:
#                 segmentValues = segment['value']
#                 segmentTensor = torch.tensor(segmentValues.to_numpy(), dtype=torch.float).unsqueeze(1)
#                 padding = 200 - len(segmentTensor)
#                 if padding > 0:
#                     segmentTensor = torch.cat((segmentTensor, torch.zeros(padding, 1)), dim=0)
#                 elif padding < 0:
#                     print('В данные зашёл сегмент неверной длины')
#                 self.cachedTensors.append(segmentTensor)

#     def __len__(self):
#         return self.segmentsCount

#     def __getitem__(self, index):
#         segmentTensor = self.cachedTensors[index]
#         return segmentTensor

In [3]:
class SecondEcgDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.segmentStarts = df.index[df['edge'] > 0].tolist()
        self.startsCount = len(self.segmentStarts)

        self.cachedTensors = []
        self.cachedLabels = []
        self._get_segment_tensor()
        self.segmentsCount = len(self.cachedTensors)
    
    def _get_segment_tensor(self):
        for index in range(self.startsCount):
            result = 0
            if index + 1 < len(self.segmentStarts):
                if self.df.at[self.segmentStarts[index], 'edge'] != 2:      # По сути условие, что длина отрезка меньше 200
                    segment = self.df.iloc[self.segmentStarts[index]:self.segmentStarts[index+1]]
                    result = 1
            elif self.df.at[self.segmentStarts[index], 'edge'] != 2:        #! Возможно это нужно будет убрать, т.к. тут в выборку может попасть отрезок длиной больше 200
                segment = self.df.iloc[self.segmentStarts[index]:]
                result = 1

            if result:
                segmentValues = segment['value']
                labelValue = (1 if segment.iloc[0]['edge'] == 3 else 0)
                segmentTensor = torch.tensor(segmentValues.to_numpy(), dtype=torch.float).unsqueeze(1)
                # print(segmentTensor.shape)
                labelTensor = torch.tensor(labelValue, dtype=torch.float).unsqueeze(0)
                # print(labelTensor.shape)
                padding = 200 - len(segmentTensor)
                if padding > 0:
                    segmentTensor = torch.cat((segmentTensor, torch.zeros(padding, 1)), dim=0)
                elif padding < 0:
                    print('В данные зашёл сегмент неверной длины')
                    exit()
                self.cachedTensors.append(segmentTensor)
                self.cachedLabels.append(labelTensor)

    def __len__(self):
        return self.segmentsCount

    def __getitem__(self, index):
        #segmentTensor = self.cachedTensors[index]
        #return segmentTensor
        return self.cachedTensors[index], self.cachedLabels[index]

In [4]:
dataset = SecondEcgDataset(df)
print(len(dataset))
#! Подумать, как размешать датасет перед разделением
trainDataset, valDataset = random_split(dataset, [int(0.7 * len(dataset)), len(dataset) - int(0.7 * len(dataset))])
# trainDataset, valDataset, testDataset = random_split(dataset, [int(0.6 * len(dataset)), int(0.2 * len(dataset)), len(dataset) - int(0.6 * len(dataset)) - int(0.2 * len(dataset))])
print(len(trainDataset))
print(len(valDataset))

trainDataloader = torch.utils.data.DataLoader(trainDataset, batch_size=10, shuffle=True)
valDataloader = torch.utils.data.DataLoader(valDataset, batch_size=10, shuffle=False)

5505
3853
1652


In [5]:
# class LSTMAutoencoder(nn.Module):
#     def __init__(self, input_size=1, hidden_size=64, num_layers=1, bidirectional=True):
#         super(LSTMAutoencoder, self).__init__()

#         # Encoder
#         self.encoder = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         # Decoder
#         self.decoder = nn.LSTM(
#             input_size=(2 * hidden_size if bidirectional else hidden_size),
#             hidden_size=input_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         self.resulter = nn.Linear(2 * input_size if bidirectional else input_size, 1)

#         # Sigmoid
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         # Encoder
#         output, _ = self.encoder(x)

#         # Decoder
#         output, _ = self.decoder(output)

#         # Resulter
#         output = self.resulter(output)

#         # Sigmoid
#         output = self.sigmoid(output) 

#         return output

In [9]:
# class LSTMAutoencoder(nn.Module):
#     def __init__(self, input_size=1, hidden_size=64, num_layers=1, bidirectional=True):
#         super(LSTMAutoencoder, self).__init__()

#         # Encoder
#         self.encoder1 = nn.LSTM(
#             input_size=input_size,
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         self.encoder2 = nn.LSTM(
#             input_size=(2*hidden_size if bidirectional else hidden_size),
#             hidden_size=hidden_size//2,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         self.encoder3 = nn.LSTM(
#             input_size=(hidden_size if bidirectional else hidden_size//2),
#             hidden_size=hidden_size//4,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         # Decoder
#         self.decoder1 = nn.LSTM(
#             input_size=(hidden_size//2 if bidirectional else hidden_size//4),
#             hidden_size=hidden_size//2,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         self.decoder2 = nn.LSTM(
#             input_size=(hidden_size if bidirectional else hidden_size//2),
#             hidden_size=hidden_size,
#             num_layers=num_layers,
#             batch_first=True,
#             bidirectional=bidirectional
#         )

#         self.resulter = nn.Linear(2 * hidden_size if bidirectional else hidden_size, 1)

#         # Sigmoid
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         # Encoder
#         output, _ = self.encoder1(x)
#         output, _ = self.encoder2(output)
#         output, _ = self.encoder3(output)

#         # Decoder
#         output, _ = self.decoder1(output)
#         output, _ = self.decoder2(output)

#         # Resulter
#         output = self.resulter(output)

#         # Sigmoid
#         output = self.sigmoid(output) 

#         return output

In [5]:
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, bidirectional=True):
        super(LSTMAutoencoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # LSTM
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )

        self.resulter = nn.Linear(2 * hidden_size if bidirectional else hidden_size, 1)

        # Sigmoid
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2 if self.bidirectional else self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2 if self.bidirectional else self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Encoder
        _, (hidden, _) = self.lstm(x, (h0, c0))
        # print(torch.cat([hidden[-2], hidden[-1]], dim=1).shape)

        # Resulter
        output = self.resulter(torch.cat([hidden[-2], hidden[-1]], dim=1))
        # print(output.shape)

        # Sigmoid
        output = self.sigmoid(output) 

        return output

In [10]:
# Параметры для создания модели
input_size = 1 # Так как ЭКГ - одномерный сигнал
hidden_size = 64 # Можно изменить в зависимости от сложности задачи и размера данных
num_layers = 1 # Можно изменить в зависимости от сложности задачи

# Создание экземпляра модели
device = ("cuda")
# model = LSTMAutoencoder(input_size, hidden_size, num_layers).to(device)
model = LSTMAutoencoder().to(device)
print(model)
# Определение функции потерь и оптимизатора
# criterion = nn.MSELoss()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

def train(trainDataloader, valDataloader, model, criterion, optimizer):
    # Обучение модели
    model.train()
    trainPbar = tqdm(trainDataloader, desc="Training")
    for inputs, targets in trainPbar:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        trainPbar.set_postfix({'Loss': f'{loss.item()}'})

    # Валидация модели
    model.eval() # Переводим модель в режим валидации
    valPbar = tqdm(valDataloader, desc="Validating")
    with torch.no_grad(): # Отключаем вычисление градиентов
        for inputs, targets in valPbar:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            valLoss = criterion(outputs, targets)
            valPbar.set_postfix({'Val_Loss': f'{valLoss.item()}'})
    
    trainPbar.close()
    valPbar.close()

    return loss.item(), valLoss.item()

LSTMAutoencoder(
  (lstm): LSTM(1, 64, batch_first=True, bidirectional=True)
  (resulter): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [11]:
ll, vl = 0, 0
for epoch in range(30):
    print(f'Epoch: {epoch} ')
    ll, vl = train(trainDataloader, valDataloader, model, criterion, optimizer)

print('---------------------------------------------------------------------------------------------')
print('Done!')

Epoch: 0 


Validating: 100%|██████████| 166/166 [00:00<00:00, 887.76it/s, Val_Loss=0.6430180072784424]


Epoch: 1 


Validating: 100%|██████████| 166/166 [00:00<00:00, 820.22it/s, Val_Loss=0.5788750648498535]


Epoch: 2 


...

Epoch: 28 


Validating: 100%|██████████| 166/166 [00:00<00:00, 748.11it/s, Val_Loss=0.028164170682430267]


Epoch: 29 


Validating: 100%|██████████| 166/166 [00:00<00:00, 833.60it/s, Val_Loss=0.02822176180779934]

---------------------------------------------------------------------------------------------
Done!


In [12]:
print('Loss: ', ll)
print('ValLoss: ', vl)

Loss:  0.028239702805876732
ValLoss:  0.02822176180779934


In [15]:
# Сохранение модели
torch.save(model.state_dict(), '../.workspace/Models/SecondStageModel.25.pt')